In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import re 
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, confusion_matrix,precision_score, roc_auc_score
from catboost import Pool,CatBoostClassifier
import os

In [2]:
df = pd.read_csv('train.csv')
df

C:\Users\aigozhiev\AppData\Local\Temp\ipykernel_15784\2339890750.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv')


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,...,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,...,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.866228,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.65161,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.809083,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,...,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.164979,Standard


In [3]:
test = pd.read_csv('test.csv')

In [4]:
def age_to_float(x):
    tmp = re.findall('-?[0-9]+', str(x))
    if len(tmp) == 0:
        return 0
    return np.float32('.'.join(tmp))

def hist_to_float(x):
    tmp = re.findall('-?[0-9]+', str(x))
    if len(tmp) == 0:
        return 0
    if len(tmp) == 1:
        return np.float32(tmp[0])
    
    ans = float(tmp[0]) + (float(tmp[1]) / 12)
    return np.float32(ans)

In [5]:
def prep(t):
    df = t.copy()
    m = {'Poor': 0,
         'Good': 2,
         'Standard': 1}
    df = df.drop(['ID', 'SSN', 'Name', 'Type_of_Loan'], axis=1)
    df['Age'] = df['Age'].apply(lambda x: re.findall('[0-9]+', x))
    df['Age'] = df['Age'].apply(lambda x: x[0])
    df['Age'] = df['Age'].astype(np.int32)
    df['Annual_Income'] = df['Annual_Income'].apply(age_to_float)
    df = df[df['Payment_Behaviour'] != '!@9#%8']
    df['Amount_invested_monthly'] = df['Amount_invested_monthly'].apply(age_to_float)
    df['Month'] = df['Month'].apply(lambda x: datetime.strptime(x, '%B').month)
    df['Monthly_Inhand_Salary'] = df.groupby('Customer_ID')['Monthly_Inhand_Salary'].transform(lambda x: x.fillna(x.mean()))
    df['Num_of_Loan'] = df['Num_of_Loan'].apply(lambda x: np.int32(re.findall('[0-9]+', x)[0]))
    df['Num_of_Delayed_Payment'].fillna('0')
    df['Num_of_Delayed_Payment'] = df['Num_of_Delayed_Payment'].apply(age_to_float)
    df['Changed_Credit_Limit'] = df['Changed_Credit_Limit'].apply(age_to_float)
    df['Outstanding_Debt'] = df['Outstanding_Debt'].apply(age_to_float)
    df['Monthly_Balance'] = df['Monthly_Balance'].apply(age_to_float)
    df['Credit_History_Age'] = df['Credit_History_Age'].apply(hist_to_float)
    df = df.drop('Customer_ID', axis=1)
    if 'Credit_Score' in df.keys():
        df['Credit_Score'] = df['Credit_Score'].map(lambda x: m[x])
    df = df.dropna()
    return df

In [6]:
set(df['Credit_Score'])

{'Good', 'Poor', 'Standard'}

In [15]:
df = prep(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90594 entries, 0 to 99998
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Month                     90594 non-null  int64  
 1   Age                       90594 non-null  int32  
 2   Occupation                90594 non-null  object 
 3   Annual_Income             90594 non-null  float32
 4   Monthly_Inhand_Salary     90594 non-null  float64
 5   Num_Bank_Accounts         90594 non-null  int64  
 6   Num_Credit_Card           90594 non-null  int64  
 7   Interest_Rate             90594 non-null  int64  
 8   Num_of_Loan               90594 non-null  int32  
 9   Delay_from_due_date       90594 non-null  int64  
 10  Num_of_Delayed_Payment    90594 non-null  float64
 11  Changed_Credit_Limit      90594 non-null  float64
 12  Num_Credit_Inquiries      90594 non-null  float64
 13  Credit_Mix                90594 non-null  object 
 14  Outsta

In [16]:
X, y = df.drop('Credit_Score', axis=1), df['Credit_Score']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)


In [18]:
cats = X.select_dtypes(include='object').keys().values

In [19]:
model = CatBoostClassifier(loss_function='MultiClass')
model.fit(X_train, y_train,
          cat_features=cats,
          eval_set=(X_test, y_test),
          verbose=False,
          use_best_model=True, 
          plot_file='plot_cbc')
#help(model.fit)

In [20]:
y_pred = model.predict(X_test)
y_pred, y_test
y_pobs = model.predict_proba(X_test)
precision_score(y_test, y_pred, average='macro')
recall_score(y_test, y_pred, average='macro')
f1_score(y_test, y_pred, average='macro')
accuracy_score(y_test, y_pred)

0.7471714774546057

In [21]:
micro_roc_auc_ovr = roc_auc_score(
    y_test,
    y_pobs,
    multi_class="ovr",
    average="macro",
)

print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")

Micro-averaged One-vs-Rest ROC AUC score:
0.89


In [ ]:
import matplotlib.pyplot as plt
x = range(len(y_test))
plt.plot(x, y_test)
plt.plot(x, y_pred)
plt.show()

In [22]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[3806, 1250,  202],
       [1138, 7557,  935],
       [  51, 1005, 2175]], dtype=int64)

In [23]:
model.fit(X, y,
          cat_features=cats,
          verbose=False,
          plot_file='plot_cbc')

In [25]:
test = prep(test)
y_final = model.predict(test)
y_final

array([[2],
       [2],
       [2],
       ...,
       [0],
       [0],
       [1]], dtype=int64)

In [26]:
m = {'Poor': 0,
         'Good': 2,
         'Standard': 1}
n = {1:'Standard',
     0: 'Poor',
     2: 'Good'}
y_final = [n[i[0]] for i in y_final]

In [27]:
y_final

['Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Good',
 'Good',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Good',
 'Poor',
 'Poor',
 'Standard',
 'Poor',
 'Good',
 'Good',
 'Good',
 'Standard',
 'Good',
 'Good',
 'Good',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Good',
 'Good',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Poor',
 'Poor',
 'Standard',
 'Good',
 'Good',
 'Good',
 'Good',
 'Poor',
 'Poor',
 'Poor',
 'Poor',
 'Standard',
 'Standard',
 'Standard',
 'Standard',
 'Poor',
 'Poor',
 'Poor',
 'Good',
 'Standard',
 'Good',
 'Good',
 'Standard',
 'Standard',
 'Standard',
 'Good',
 'Poor',
 'Poor',
 'Poor',